In [ ]:
#SCRAPE MULTIPLE FINANCIAL PARAMETERS FROM SINGLE PAGE / V2 / Faster scrolling using drag&drop apprach
#https://stackoverflow.com/questions/62119348/how-to-scroll-horizontally-using-selenium-chromedriver-in-python

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
#ToDo
#Add Support for balanse-sheet,cashflow-statement,financial-ratios and other

In [ ]:
class ScrapeMacrotrend:
    
    def __init__(self, ticker_name):
        
        self.start = time.time() #start timer
        self.ticker_name = ticker_name #ticker or company name
        
        #innitialize and set chrome-webdriver options
        self.chrome_options = Options()
        #self.chrome_options.add_argument("--window-size=1000,1080")
        #self.chrome_options.add_argument("--headless")
        
        self.driver = webdriver.Chrome("G:\My Drive\Investing\Programming\chromedriver.exe", options=self.chrome_options)
        self.driver.implicitly_wait(5)
        self.driver.maximize_window()
        self.base_url = "https://www.macrotrends.net/"
        
        self.ad_clicked = False
        self.company_url = self.getCompanyURL()
        
        #scrape main pages
        self.income_statement = self.runScraper(self.company_url + "/income-statement?freq=Q")
        self.balance_sheet = self.runScraper(self.company_url + "/balance-sheet?freq=Q")
        self.cash_flow_statement = self.runScraper(self.company_url + "/cash-flow-statement?freq=Q")
        self.financial_ratios = self.runScraper(self.company_url + "/financial-ratios?freq=Q")
        
        self.driver.close() #close chrome-webdriver
        self.end = time.time() #stop timer
        print('Complete time: {}s'.format(self.end - self.start))

        
        
        
    def getCompanyURL(self): #this function finds the company url f.e. https://www.macrotrends.net/stocks/charts/AAPL/apple
        self.driver.get(self.base_url)
        self.search_box = self.driver.find_element_by_css_selector("input.js-typeahead")
        self.search_box.send_keys(self.ticker_name)
        
        #get company url
        self.company_url = self.driver.find_element_by_xpath("//li[1]/a") #xpath element of the company urls
        self.company_url = self.company_url.get_attribute('href') #get the href-value
        self.company_url = self.company_url.rsplit("/", 1)[0] #split url to remove unnecesery data
        
        return self.company_url
        
        
    def runScraper(self, url):
        
        #scraper variables
        self.periods_table_prev_last_elem = -1 #previously(before scrolling) last scraped element of periods-table
        self.periods_table_current_last_elem = 0 #current(after scrolling) last scraped element of periods-table
        self.main_table_prev_last_elem = -1  #previously(before scrolling) last scraped element of main-table
        self.main_table_current_last_elem = 0 #current(after scrolling) last scraped element of main-table

        #data-storring variables
        self.periods_table_data = [] #store data from periods table (annual or quarter)
        self.main_table_data = [] #store data from the main-table like revenue, 
        self.all_scraped_data = pd.DataFrame()
        
        self.driver.get(url)

        #find advertise window button and click it
        if self.ad_clicked == False:
            self.ad_button = self.driver.find_element_by_xpath('//button[@class="Button__StyledButton-a1qza5-0 cUAUIG"]')
            self.ad_button.click()
            self.ad_clicked = True


        self.horizontal_bar_width = self.driver.find_element_by_id('jqxScrollOuterWraphorizontalScrollBarjqxgrid').rect['width']
        self.slider = self.driver.find_element_by_id('jqxScrollThumbhorizontalScrollBarjqxgrid')   

        #scroll table from left to right and scrape data
        while (self.periods_table_prev_last_elem != self.periods_table_current_last_elem): #and (self.main_table_prev_last_elem != self.main_table_current_last_elem):
            

            #set the last element from the last scrape as previous one
            self.periods_table_prev_last_elem = self.periods_table_current_last_elem
            self.main_table_prev_last_elem = self.main_table_current_last_elem

            # self.periods_table = self.driver.find_element_by_xpath('//*[@id="columntablejqxgrid"]') 
            self.periods_table = self.driver.find_element_by_xpath('//*[@id="contentjqxgrid"]/div[1]') #this is the header row in the table, containing the dates.
            # print(self.periods_table.text)
            #parameters_table = self.driver.find_element_by_xpath('//*[@id="contentjqxgrid"]/div[2]')
            self.periods_table_split = self.periods_table.text.splitlines() #split the periods to array
            self.periods_table_current_last_elem = self.periods_table_split[-1] #get the last element from the table
            # print(len(self.periods_table_split), "periods")
            # print('last-element={}'.format(self.periods_table_current_last_elem))

            # main_table = self.driver.find_elements_by_xpath('//div[@role="row"]') #get the main table
            main_table = self.driver.find_elements_by_xpath('//*[@id="contentjqxgrid"]/div[2]/div/div') #get the main table
            self.main_table_current_last_elem = main_table[0].text.splitlines()[-1]
            # print(len(main_table[0].text.splitlines()), "main")
            #print(self.main_table_current_last_elem)
            
            #this is the innitial position of the table, no scrolling has been executed yet
            if self.periods_table_prev_last_elem == 0: 
                #add scraped elements of periods table to a temp-variable 
                self.periods_table_data.extend(self.periods_table_split) 
                
                #add the scraped elements of main-table to a temp-variable
                for i in range (len(main_table)): #add all parameter to table
                    main_table_split = main_table[i].text.splitlines()
                    self.main_table_data.append(main_table_split)

            else:
                #find the previous last element when the periods-table is scrolled to the right
                self.periods_table_prev_last_elem_index = self.periods_table_split.index(self.periods_table_prev_last_elem)
                #add the new elements after the periods-table is scrolled to the right
                self.periods_table_data.extend(self.periods_table_split[self.periods_table_prev_last_elem_index+1:]) 
                # print(self.periods_table_split)
                
                #find the previous last element when the main-table is scrolled to the right
                self.main_table_prev_last_elem_index = main_table[0].text.splitlines().index(self.main_table_prev_last_elem) 
                # print(main_table[0].text.splitlines())
                #add the new elements after the main-table is scrolled to the right
                for row in range (len(main_table)): #add the new data to the table after scrolling table to the right
                    main_table_split = main_table[row].text.splitlines()
                    self.main_table_data[row].extend(main_table_split[self.main_table_prev_last_elem_index+1:])
                


                # raise SystemExit("Stop right there!")

            # Ensure the slider is in view
            self.slider.location_once_scrolled_into_view
            #scroll table to the right
            ActionChains(self.driver).click_and_hold(self.slider).move_by_offset(self.horizontal_bar_width/5, 0).release().perform()

        
        #add the scraped data to pandas dataframe
        for i in range (len(self.main_table_data)):
            #create temp dataframe with scraped data
            # print(self.main_table_data[i], self.periods_table_data)
            data = pd.DataFrame([self.main_table_data[i][1:]], columns=self.periods_table_data[1:], index=[self.main_table_data[i][0]]) 
            
            #append temp-dataframe row to global dataframe
            self.all_scraped_data = pd.concat([self.all_scraped_data, data]) 

        
        #return scraped data
        return self.all_scraped_data
        
# apple = ScrapeMacrotrend("https://www.macrotrends.net/stocks/charts/AAPL/apple/income-statement?freq=Q")
# apple = ScrapeMacrotrend("aapl")
apple = ScrapeMacrotrend("apple")

In [ ]:
apple.cash_flow_statement

In [ ]:
company_data

In [ ]:
#remove "$" from the values
def fixData(input_data):
    
    for i in range(len(input_data)):
        input_data[i]=input_data[i].replace("$","")
        input_data[i]=input_data[i].replace(",",".")
        # try:
        #     input_data[i] =  float(input_data[i])
        #     # print(type(input_data[i]), input_data[i])
        # except:
        #     pass
        #     # print(input_data[i],"can't convert to float")
        
        try:
            input_data[i] = pd.to_numeric(input_data[i])
            # print(type(input_data[i]), "converted to numeric")

        except:
            pass
            # print("Can't convert to numeric")
        
    return input_data
    

c_data = company_data.copy()
c_data.apply(fixData)
# c_data.apply(pd.to_numeric)
c_data = c_data.T #transpose the table
# c_data.index.to_datetime()
c_data.index = pd.to_datetime(c_data.index) #change index data-type to datetime.
c_data

# c_data["Revenue"] = pd.to_numeric(c_data["Revenue"]) #convert data-type from string to number
# c_data["Gross Profit"] = pd.to_numeric(c_data["Gross Profit"]) #convert data-type from string to number
c_data["Revenue"].plot()
c_data["Gross Profit"].plot()
plt.show()

# company_data
#DataFrame.apply(func, axis=0, raw=False, result_type=None, args=(), **kwargs)
# df = pd.DataFrame({
#    'pig': [20, 18, 489, 675, 1776],
#    'horse': [4, 25, 281, 600, 1900]
#    }, index=[1990, 1997, 2003, 2009, 2014])
# lines = df.plot.line()
# df

In [ ]:
company_data.T.columns